In [1]:
import logging
logging.getLogger('tensorflow').disabled = True
# import utils
import data_preparation
import os
import params
import numpy as np
import model_lib
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import utils
from tqdm import trange
tfd = tfp.distributions
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
train_size = 0
for m in params.brand_models:
    train_size += len(os.listdir(os.path.join(params.patches_dir, 'train', m)))
num_train_steps = (train_size + params.BATCH_SIZE - 1) // params.BATCH_SIZE

In [4]:
kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) /  # pylint: disable=g-long-lambda
                        tf.cast(10 * num_train_steps, dtype=tf.float32))
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(3, (5, 5), 
                           padding='same'),
    tfp.layers.Convolution2DFlipout(96, 
        kernel_size=7, strides=2, padding='SAME', 
        kernel_divergence_fn=kl_divergence_function,
        activation=tf.nn.selu),
    tf.keras.layers.MaxPool2D(
        pool_size=[3, 3], strides=2,
        padding='SAME'),
    tfp.layers.Convolution2DFlipout(
        64, kernel_size=5, strides=1,
        padding='SAME', 
        kernel_divergence_fn=kl_divergence_function,
        activation=tf.nn.selu),
    tf.keras.layers.MaxPool2D(
        pool_size=[3, 3], strides=1,
        padding='SAME'),
    tfp.layers.Convolution2DFlipout(
        128, kernel_size=1, 
        strides=1, padding='SAME', 
        kernel_divergence_fn=kl_divergence_function,
        activation=tf.nn.selu),
    tf.keras.layers.MaxPool2D(
        pool_size=[3, 3], strides=2,
        padding='SAME'),
    tf.keras.layers.Flatten(),
    tfp.layers.DenseFlipout(
        len(params.brand_models), kernel_divergence_fn=kl_divergence_function)
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
              experimental_run_tf_function=False)
model.build(input_shape=[None, 256, 256, 1])

# Loads the weights
model.load_weights('./ckpts/10_num_batches/')

In [6]:
names = [layer.name for layer in model.layers 
        if 'flipout' in layer.name]
# dense_flipout/kernel_posterior_loc:0
qm_vals = [layer.kernel_posterior.mean() 
        for layer in model.layers
        if 'flipout' in layer.name]

qs_vals = [layer.kernel_posterior.stddev() 
        for layer in model.layers
        if 'flipout' in layer.name]

utils.plot_weight_posteriors(names, qm_vals, qs_vals, 'trained_weight.png')
logging.info("mean of mean is {}, mean variance is {}".
              format(tf.reduce_mean(qm_vals[0]),
              tf.reduce_mean(qs_vals[0])))

saved trained_weight.png


In [4]:
utils.set_logger('evaluate.log')
num_monte_carlo = 30

# Create the input data pipeline
logging.info("Creating the dataset...")
test_iterator = data_preparation.build_dataset('test')

# Define the model
logging.info("Creating the model...")
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

@tf.function
def test_step(images, label):
    with tf.GradientTape() as tape:
        logits = model(images)
        neg_log_likelihood = tf.keras.losses.categorical_crossentropy(labels, logits, from_logits=True)
        kl = sum(model.losses)
        loss = neg_log_likelihood + kl

    test_loss.update_state(loss)  
    test_accuracy.update_state(labels, logits)  

test_size = 0
for m in params.brand_models:
    test_size += len(os.listdir(os.path.join(params.patches_dir, 'test', m)))
num_test_steps = (test_size + params.BATCH_SIZE - 1) // params.BATCH_SIZE

Creating the dataset...
Creating the model...


In [5]:
for step in trange(num_test_steps):
    images, labels = test_iterator.get_next()
    test_step(images, labels)

    if step % 50 == 0:
        probs, heldout_log_prob = utils.compute_probs(model, images)
        logging.info(' ... Held-out nats: {:.3f}\n'.format(heldout_log_prob))
        utils.plot_heldout_prediction(images, labels.numpy(), probs,
                              fname='step{}_pred.png'.format(step),
                              title='mean heldout logprob {:.2f}'
                              .format(heldout_log_prob.numpy()))                                                 

logging.info('test loss: {:.3f}, test accuracy: {:.3%}\n'.format(test_loss.result(),
                                                    test_accuracy.result()))

  0%|          | 0/276 [00:00<?, ?it/s] ... Held-out nats: 0.077



 ... Held-out nats: 0.077


/home/pan/anaconda3/envs/bnn/lib/python3.7/site-packages/matplotlib/text.py:1165: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if s != self._text:
  1%|          | 3/276 [00:08<25:22,  5.58s/it]

saved step0_pred.png


 18%|█▊        | 49/276 [00:11<00:18, 12.16it/s] ... Held-out nats: 0.067



...

 90%|█████████ | 249/276 [00:48<00:02, 12.77it/s] ... Held-out nats: 0.136



 ... Held-out nats: 0.136


 92%|█████████▏| 253/276 [00:53<00:14,  1.62it/s]

saved step250_pred.png


100%|██████████| 276/276 [00:55<00:00,  4.95it/s]
test loss: 124.962, test accuracy: 92.861%

